In [209]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [210]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")

uv_train, uv_test = train_test_split(user_vectors, test_size=0.2, random_state=8)

ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [211]:
encoder = LabelEncoder()
t_encoder = LabelEncoder()
final_vectors = uv_train.copy()
test_vectors = uv_test.copy()
x = encoder.fit(final_vectors["username"])
t = t_encoder.fit(test_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
test_enc = t.transform(test_vectors["username"])

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc
test_vectors["username"] = test_enc
#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
test_arrays = test_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])
#final_vectors
uv_train
#test_arrays
#test_vectors
#uv_test

,username,experience,gender,generation,fav_anime_period,Slice,Dementia,Arts,Adventure,Vampire,Comedy,Shounen,Sports,Power,Military,Space,Mecha,Music,Supernatural,Mystery,Horror,Kids,Cars,Harem,School,Shoujo,Parody,Psychological,Action,Sci-Fi,Game,Fantasy,Martial,Romance,Ai,Life,Drama,of,Ecchi,Police,Historical,Super,Magic,Thriller,Seinen
7053,nemeklau,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4363,TheoriginalMadD,3,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8451,Napalmitos,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1478,MrYosuko,0,1,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8047,GenkaiShinigami,2,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5480,NirvashThurston,2,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6995,Harmleikur,0,0,2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2181,Xanquis,2,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2033,Ali_Mohamd,3,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Compute K-Nearest Neighbours

In [212]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours(user):
    #user = np.array([7537, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data (training)
    n = train.kneighbors([user], return_distance = False)
    return n

#returns a dataframe of users that were in k-nearest neighbours list 
def sim_frame(neighbours):
    sim_names = []
    #neighbours = neighbours()

    for i in neighbours[0]: #loop through nested array
        #print(final_arrays[i][0])
        name = x.inverse_transform([final_arrays[i][0]]) #find usernames of similar users
        sim_names.append(name)

    #build new dataframe of neighbours
    neighbour_frame = pd.DataFrame()
    for i in users.columns:
        neighbour_frame[i] = None

    for name in sim_names:
        user_row = users.loc[users["username"]==name[0]]
        neighbour_frame = pd.concat([neighbour_frame, user_row])

    pd.options.display.max_rows = 999
    pd.set_option('display.max_columns', 500)
    return neighbour_frame

neighbour_ids = neighbours(np.array([6490, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0]))
#print(neighbour_ids)
neighbour_frame = sim_frame(neighbour_ids)
neighbour_frame
#final_vectors.loc[final_vectors["username"]==7537]
#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
7053,362316,nemeklau,162,Active,Female,1996,4916,Gen Z,"['Action', 'Mystery', 'Supernatural']",New-Gen
7111,3964665,XleamoriX,260,Active,Female,1999,4911,Gen Z,"['Action', 'Supernatural', 'Mystery']",New-Gen
1008,5998161,yNatsu,261,Active,Male,2001,5803,Gen Z,"['Action', 'Mystery', 'Supernatural']",New-Gen
3936,3715413,hallyson,243,Active,Male,1997,5125,Gen Z,"['Mystery', 'Supernatural', 'Action']",New-Gen
7423,189319,Yuki95,189,Active,Female,1995,5961,Gen Z,"['Action', 'Mystery', 'Supernatural']",Classic


Create anime recommendations

In [213]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours, media_type):
    #Get top rated TV shows or movies from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        tv_shows = user_ratings[user_ratings["type"]=="TV"]
        movies = user_ratings[user_ratings["type"]=="Movie"]

        if media_type == "TV":
            show_ratings = tv_shows
        else:
            show_ratings = movies
        
        #There may be 0 movies in the user's top 10 list in which case we skip that user's recommendationan
        if len(show_ratings) > 1:
            top_rated = show_ratings.iloc[0]["title"]
            suggestions.append(top_rated)
            #print(top_rated)

    if len(suggestions)==0:
        print("No movies to recommend :/")
    
    return suggestions

recommendations(neighbour_frame, "TV")
#ratings

['Uta no☆Prince-sama♪ Maji Love 2000%',
 'Death Parade',
 'Code Geass: Hangyaku no Lelouch',
 'Steins;Gate',
 'Fullmetal Alchemist: Brotherhood']

Based on model fit with training data (K-Neighbours), evaluate recommended anime for users in test set


In [214]:
real_ratings = pd.read_csv(r"D:\dataset\animelists_cleaned.csv")
real_anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

In [215]:
def evaluation():
    #Create kneighbours dataframe for all users in the dataset
    percentage_scores = []
    for user in test_arrays:
        user = np.array(user)
        #print([user[0]])
        username = t.inverse_transform([user[0]])[0]
        #print(username)
        user_neighbours = neighbours(user)
        nframe = sim_frame(user_neighbours)
        #print(user_neighbours)
        user_recommendations = recommendations(nframe, "TV")
        #print(nframe["username"], user_recommendations)
        rt = real_ratings.loc[real_ratings["username"]==username]

        score = 0
        #Check whether recommended shows are present in original ratings table
        for show in user_recommendations:
            a_id = int(real_anime.loc[real_anime["title"]==show]["anime_id"])
            #u_ratings = real_ratings.loc[real_ratings["username"]==username]

            if int(a_id) in list(rt["anime_id"]): #if user watched the recommended show
                rating_row = real_ratings.loc[(real_ratings["username"]==username)&(real_ratings["anime_id"]==a_id)]
                score+=1
                if int(rating_row["my_score"])>=7:#user "likes" the show if they rated it higher than or equal to 7
                    score+=1 #add an extra point if the user liked the shower

        score = (score/10)*100 #max number of points is 10
        percentage_scores.append(score)
        #if user liked show --> give double points
        print(f"Score for user {username}: {score}%")

    percentage_scores = np.array(percentage_scores)
    average_score = np.average(percentage_scores)
    print(f"Average recommendation score for Test users is {average_score}%")
    return average_score

evaluation()
#include analysis of distance metric between neighbours e.g. cosine, euclidean 

Score for user Herdrocks: 40.0%
Score for user NikuTaicho: 50.0%
Score for user diwwi: 20.0%
Score for user Mivako: 30.0%
Score for user Akimaeru: 0.0%
Score for user Itachi2099: 40.0%
Score for user Arrakhis: 20.0%
Score for user covan: 80.0%
Score for user Rodrigo_Sousa: 40.0%
Score for user Cheesecakery: 40.0%
Score for user Kaomi: 40.0%
Score for user CruelAngel: 60.0%
Score for user alba2darkangel: 50.0%
Score for user yukari_1998: 80.0%
Score for user MartyrrSenpai: 50.0%
Score for user Imaginaryfriend: 20.0%
Score for user anowi: 50.0%
Score for user Swoolie: 40.0%
Score for user Spiddler: 50.0%
Score for user schmyke: 40.0%
Score for user Blahblah692: 60.0%
Score for user sapphireblue116: 80.0%
Score for user Qc_Cerberus: 20.0%
Score for user sasorinor: 40.0%
Score for user RyHawk: 90.0%
Score for user BiribiriRoan: 40.0%
Score for user EnigmaticTwin: 80.0%
Score for user _gallo_: 20.0%
Score for user Toru-Mizushima: 60.0%


KeyboardInterrupt: 

Function that converts user information from website into vector